In [7]:
import pyximport; pyximport.install()
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import set_types
import monte_carlo
from tqdm import tqdm

%matplotlib inline

# Reproduce Tristan Barnett Results

## Standard scoring systems

In [2]:
spw_1, spw_2 = 0.77, 0.73

final_set_no_tb = lambda win_probs: set_types.play_standard_set(win_probs, has_tiebreak=False)
final_set_with_tb = lambda win_probs: set_types.play_standard_set(win_probs, has_tiebreak=True)

system_4 = lambda spw_1, spw_2: set_types.best_of(
    set_types.play_standard_set, [spw_1, spw_2], 
    5, final_set_fun=final_set_no_tb)

system_5 = lambda spw_1, spw_2: set_types.best_of(
    set_types.play_standard_set, [spw_1, spw_2], 
    3, final_set_fun=final_set_no_tb)

system_6 = lambda spw_1, spw_2: set_types.best_of(
    set_types.play_standard_set, [spw_1, spw_2], 
    5, final_set_fun=final_set_with_tb)

system_7 = lambda spw_1, spw_2: set_types.best_of(
    set_types.play_standard_set, [spw_1, spw_2], 
    3, final_set_fun=final_set_with_tb)

system_8 = lambda spw_1, spw_2: set_types.best_of(
    set_types.play_standard_set, [spw_1, spw_2], 
    3, final_set_fun=set_types.super_tb)

system_9 = lambda spw_1, spw_2: set_types.best_of(
    lambda win_probs: 
    set_types.play_standard_set(win_probs, service_game_ad=False), 
    [spw_1, spw_2], 
    3, final_set_fun=set_types.super_tb)

all_systems = {4: system_4, 5: system_5, 6: system_6, 7: system_7,
               8: system_8, 9: system_9}

In [10]:
num_trials = int(1e5)

system_results = dict()

for num, fn in tqdm(all_systems.items()):
    
    num_pts = list()
    p1_won = list()
    
    for i in range(num_trials):
        
        if np.random.choice([0, 1]) == 0:
        
            result = fn(spw_1, spw_2)

            p1_won.append(result.final_score[0] > result.final_score[1])
            
        else:
            
            result = fn(spw_2, spw_1)
            
            p1_won.append(result.final_score[0] < result.final_score[1])
            
        num_pts.append(result.total_points)
        
    better_won = np.mean(p1_won)
    mean_pts = np.mean(num_pts)
    std_pts = np.std(num_pts)
    pct_98 = np.percentile(num_pts, 98)
        
    system_results[num] = {'better_won': better_won,
                           'mean_pts': mean_pts,
                           'std_pts': std_pts,
                           '98th': pct_98}

100%|██████████| 6/6 [13:30<00:00, 126.81s/it]


In [11]:
pd.DataFrame(system_results)

,4,5,6,7,8,9
98th,579.000000,474.000000,385.000000,242.000000,186.000000,173.000000
better_won,0.725280,0.687740,0.708620,0.668100,0.655360,0.659150
mean_pts,290.334430,192.188900,272.071440,166.297450,142.860700,131.460450
std_pts,100.129184,92.925892,60.850896,40.227911,21.698239,20.510174
